In [1]:
from pulp import *
from itertools import product
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from tqdm import tqdm

#### Sets

In [2]:
%%time
#import files from github

V = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/V.csv') #vehicle types
F = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/F.csv') #fuel types
E = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/E.csv') #driving env
R = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/R.csv') #counties
M = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/M.csv') #charging stations
S = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/S.csv') #states

F['fuel_type'] = F['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))

CPU times: user 80.6 ms, sys: 10.1 ms, total: 90.7 ms
Wall time: 1.99 s


In [3]:
VEHICLE_TYPES = list(V['vehicle_type'])
FUEL_TYPES = list(F['fuel_type'])
DRIVING_ENV = list(E['driving_environment'])
COUNTIES = list(R['county'])
CHARGING_STATIONS = list(M['filling_stations'])
STATES = list(S['state'])

#### Parameters
Parameters describe objects statically, and is constant in a single simulation. Parameters are only changed to adjust model behaviour.

In [4]:
%%time
#EF(f,s): Emission factor for fuel type f in state s, in gallons/mile  
EF = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/EF(f%2Cs).csv')
EF['fuel_type'] = EF['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#FE(v,f):Average fuel economy for vehicle type v using fuel f
FE = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/FE(v%2Cf).csv')
FE['fuel_type'] = FE['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#C(f): Cost of fuel type f 
C = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/C(F).csv')
#CC (v,s): Capital cost of vehicle type v in state s
CC = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/CC(v%2Cs).csv') 
CC['yearly_cost']=np.where(CC['vehicle_type']=='BEV', CC['cost_minus_rebate']/8.5, CC['cost_minus_rebate']/5.5)
#BEVs are paid off in approx. 8-9 years, SIDI ICE and FFV are paid off in 5-6 years.
#CG: cost of fuel/gallon
CG = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/CG(F).csv')
CG['fuel_type'] = CG['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
#D: Emission decrease goals per year
D = 0.25
#W(s):Current yearly GHG emissions per state
#W = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/W(s).csv')
#TM (v, f, r): Total miles for vehicle v using fuel f in county r
TM = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/TM(f%2Cs).csv')
# # TM.drop(['household_income_ID'],axis=1,inplace=True)
#N(r): Average income per county  
N = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/N(r).csv')
# N.drop(['household_income_ID'],axis=1,inplace=True)
#B(r) county + state linking table
B = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/B(r).csv')
#CF(v,f): Fuel consumption for vehicle type v using fuel f (1/fuel economy)
CF = FE
CF['fuel_consumption'] = (1 / CF['fuel_economy'])
T = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/T(r).csv')
T['total_vehicles_registered'] = T['total_vehicles_registered'].apply(lambda x: x.replace(',',''))
T['total_vehicles_registered'] = T['total_vehicles_registered'].apply(pd.to_numeric)
W_param = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/W_county_param.csv')

CPU times: user 378 ms, sys: 118 ms, total: 495 ms
Wall time: 4.44 s


# Variables
Variables represent a model state and may change during simulation.

In [5]:
v = V['vehicle_type']
f = F['fuel_type']
r = R['county']

def variable_n():
    '''
    n(r,v,f) total optimal count of vehicle v using fuel type f in county r
    '''
    n = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    n['count'] = 0
    
    bev_elec = n[(n['vehicle_type']== 'BEV') & (n['fuel_type']== 'Electricity')]
    gas_e10 = n[(n['vehicle_type']== 'SIDI_ICE') & (n['fuel_type']== 'E10')]
    ffv_e85 = n[(n['vehicle_type']== 'FFV') & (n['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county','vehicle_type'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

def variable_fc():
    '''
    fc(r,v,f) Total fuel consumption by vehicle v using fuel type f in county r
    '''
    fc = pd.DataFrame(list(product(r,v,f)), columns=['county','vehicle_type','fuel_type'])
    fc['fuel_consumption'] = 0
    
    bev_elec = fc[(fc['vehicle_type']== 'BEV') & (fc['fuel_type']== 'Electricity')]
    gas_e10 = fc[(fc['vehicle_type']== 'SIDI_ICE') & (fc['fuel_type']== 'E10')]
    ffv_e85 = fc[(fc['vehicle_type']== 'FFV') & (fc['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

def variable_oc():
    '''
    oc(r,v,f) Annual operating cost for vehicle v using fuel type f in county r 
    '''
    oc = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    oc['operating_cost'] = 0
    
    bev_elec = oc[(oc['vehicle_type']== 'BEV') & (oc['fuel_type']== 'Electricity')]
    gas_e10 = oc[(oc['vehicle_type']== 'SIDI_ICE') & (oc['fuel_type']== 'E10')]
    ffv_e85 = oc[(oc['vehicle_type']== 'FFV') & (oc['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result 

def variable_tac():
    '''
    tac(r,v,f) Total annual cost of vehicle v using fuel type f in county r 
    '''
    tac = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    tac['total_annual_vehicle_cost'] = 0
    
    bev_elec = tac[(tac['vehicle_type']== 'BEV') & (tac['fuel_type']== 'Electricity')]
    gas_e10 = tac[(tac['vehicle_type']== 'SIDI_ICE') & (tac['fuel_type']== 'E10')]
    ffv_e85 = tac[(tac['vehicle_type']== 'FFV') & (tac['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

def variable_ce():
    '''
    ce(r,v,f)  GHG emission per year of vehicle v using fuel type f in county r 
    '''
    ce = pd.DataFrame(list(product(r,v,f)), columns=['county', 'vehicle_type','fuel_type'])
    ce['emission_per_year'] = 0
    
    bev_elec = ce[(ce['vehicle_type']== 'BEV') & (ce['fuel_type']== 'Electricity')]
    gas_e10 = ce[(ce['vehicle_type']== 'SIDI_ICE') & (ce['fuel_type']== 'E10')]
    ffv_e85 = ce[(ce['vehicle_type']== 'FFV') & (ce['fuel_type']== 'E85')]

    result = pd.concat([bev_elec,gas_e10,ffv_e85])
    result.sort_values(by=['county'],inplace=True)
    result.reset_index(drop=True, inplace=True)
    return result

n = variable_n()
fc = variable_fc()
tac = variable_tac()
oc = variable_oc()
ce = variable_ce()

## IEMS 394 - Biofuels Optimisation Model

Running list of assumptions:
- Some county name recurr within the set of states we have selected. Such counties have had their respective 2-letter state code appended to their name. They are enumerated below.
    1. Orange - CA,TX 
    2. Cass - MN, TX
    3. Lake - CA, MN
    4. Trinty - CA, TX
    5. Houston - MN, TX
    6. Polk - MN,TX
    7. Brown - MN,TX
    8. Clay - MN, TX
    9. Jackson - TX,MN
    10. Washington - MN, TX
    11. Martin - MN,TX
    
- future proofing changes:
    - Decision variable `n` assumes that vehicle types use a single type of fuel (it's not written in at all)

In [6]:
n1 = n.merge(B)
n2 = n1.merge(CG)
n3 = n2.merge(oc)
n4 = n3.merge(ce)
n5 = n4.merge(fc)
n6 = n5.merge(tac)
n6 = n6.sort_values(by=['county','vehicle_type'], ascending=True)
n6.reset_index(drop=True, inplace=True)

In [7]:
N_B = B.merge(N)
number_one = N_B.merge(TM, left_on=['household_income_ID','state'],right_on=['household_income_ID','state'])
number_one.drop(['household_income','household_income_ID'],axis=1,inplace=True)
n6 = n6.merge(number_one)
total_miles_df = n6[['county','vehicle_type','annual_miles_driven']]

# Parameters

In [8]:
EF_param = {k: f.groupby('fuel_type')['emission_factor'].apply(list).to_dict()
     for k, f in EF.groupby('state')}

FE_param = {k: f.groupby('fuel_type')['fuel_economy'].apply(list).to_dict()
     for k, f in FE.groupby('vehicle_type')}

C_param = {k: f.groupby('fuel_type')['fuel_cost_per_mile'].apply(list).to_dict()
     for k, f in C.groupby('state')}
# CC_param = {k: f.groupby('vehicle_type')['cost_minus_rebate'].apply(list).to_dict()
#      for k, f in CC.groupby('state')}
improve = CC.merge(B)
CC_param = {k: f.groupby('vehicle_type')['yearly_cost'].apply(list).to_dict()
     for k, f in improve.groupby('county')}
CC_param

CG_param = {k: f.groupby('fuel_type')['fuel_cost_per_gal'].apply(list).to_dict()
     for k, f in CG.groupby('state')}
D_param = 0.80

TM_param = {k: f.groupby('vehicle_type')['annual_miles_driven'].apply(list).to_dict()
     for k, f in total_miles_df.groupby('county')}
# CF_param = {k: f.groupby('fuel_type')['fuel_economy'].apply(list).to_dict()
#      for k, f in CF.groupby('vehicle_type')}
CFnew = CF.merge(n6,left_on="vehicle_type",right_on="vehicle_type")
CFnew.sort_values(by=['county'],inplace=True)
CFnew.reset_index(drop=True, inplace=True)
CFnew = CFnew[['county','vehicle_type','fuel_consumption_x']]
CFnew_param = {k: f.groupby('vehicle_type')['fuel_consumption_x'].apply(list).to_dict()
     for k, f in CFnew.groupby('county')}

CGnew_param = n6[['county','vehicle_type','fuel_cost_per_gal']]
CGnew_param = {k: f.groupby('vehicle_type')['fuel_cost_per_gal'].apply(list).to_dict()
     for k, f in CGnew_param.groupby('county')}
     

T_param = T.groupby('county')['total_vehicles_registered'].apply(list).to_dict()
W_param = W_param.groupby('county')['emissions_per_county'].apply(list).to_dict()

# Decision Variables

In [9]:
#n (v, f, r) 
#Projected number of vehicles of vehicle type v using fuel type f that should be in county r
n = LpVariable.dicts("vehicle_count", (COUNTIES, VEHICLE_TYPES), 0)

#fc(r,v,f) 
#Total fuel consumption by vehicle v using fuel type f in county r
# fc = LpVariable.dicts('total_fuel_use',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')
fc = LpVariable.dicts("total_fuel_use", (COUNTIES, VEHICLE_TYPES), 0)

#oc(r,v,f) 
#Operating cost per mile for vehicle v using fuel type f in county r 
# oc = LpVariable.dicts('operating_cost_permile',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')
oc = LpVariable.dicts("annual_operating_cost", (COUNTIES, VEHICLE_TYPES), 0)

#tac(r,v,f) 
#Total annual cost of vehicle v using fuel type f in county r 
# tac = LpVariable.dicts('total_annual_cost',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = 'Integer')
tac = LpVariable.dicts("total_annual_cost", (COUNTIES, VEHICLE_TYPES), 0)

#ce(r,v,f)  
#GHG emission per year of vehicle v using fuel type f in county r
# ce = LpVariable.dicts('GHG_emissions_new',[(r,f) for r in COUNTIES for f in VEHICLE_TYPES], cat = "Integer")
ce = LpVariable.dicts("GHG_emissions_new", (COUNTIES, VEHICLE_TYPES), 0)

# Objective Function

In [10]:
#SET PROBLEM VARIABLE
prob = LpProblem('Biofuels',LpMinimize)
#OBJECTIVE FUNCTION: minimise cost, determine car allocation (mixed integer)
prob += lpSum(((CFnew_param[r][f][0] * CGnew_param[r][f][0] * TM_param[r][f][0]) + CC_param[r][f][0]) * n[r][f] for f in VEHICLE_TYPES for r in COUNTIES)

#### Cost Commentary:

Our model currently says we need to spend 434 billion to change each county's split to an optimal which reduces GHG by 25% for each state. However, this assumes there aren't any vehicles on the ground - in other words, it's a raw cost as if we bought all these vehicles. In reality, there's already some (inefficient/suboptimal) split that exists. Our cost should reflect that change (suboptimal split ---> optimal split)

So let's say if Alameda has (2 BEV, 18 FFV, 80 SIDI ICE), and the optimal split is (30 BEV, 28 FFV, 42 SIDI ICE), our cost parameter should account for the changes (+28 BEV, +10 FFV, -38 SIDI ICE)

In order to properly compute the split differential, we would need to know the county-level data for vehicle split. Currently, we have total counts, as well as EV registration data. By assuming a percentage (%) for FFVs, we can compute all three values in order to create a rough split.

For a client who cares about finances (Dept of Energy, EPA / any govt body) could take one look at this cost vector and trash the whole model. Even if what a viewer cares about is the optimal vehicle split, there is a chicken-and-egg problem with chasing cost reductions. Counties with close-to-optimal splits will spend significantly less, and as a result different optimalities will be found.

# Constraints

In [11]:
v = V['vehicle_type']
f = F['fuel_type']
r = R['county']
CE = variable_ce()
ce_update = CE.merge(B)
#ce = {k: f.groupby('county')['emission_per_year'].apply(list).to_dict()
     #for k, f in ce_update.groupby('state')}

In [12]:
#Constraint 0: Non-negative vehicles assigned to each county r 
# for r in COUNTIES:
#     prob += lpSum(n[r][f] for f in VEHICLE_TYPES) >= 0
for r in COUNTIES:
    for f in VEHICLE_TYPES:
        prob += n[r][f] >= 0

In [13]:
Nnew = N.merge(B)
EFnew = pd.merge(TM, Nnew,  how='left', left_on=['state','household_income_ID'], right_on = ['state','household_income_ID']).dropna()
EFnew = EFnew.merge(EF).merge(FE)
EFnew.sort_values(by=['county'],inplace=True)
EFnew.reset_index(drop=True, inplace=True)
EFnew['TotalEmission'] = EFnew['annual_miles_driven']*EFnew['emission_factor']
EFnew
EFnew_param = {k: f.groupby('county')['TotalEmission'].apply(list).to_dict()
      for k, f in EFnew.groupby('county')}

#Constraint 1: Annual emission by total of vehicle v in county f with a given fuel ce(r,f) equals emission per mile * total miles driven 

for r in COUNTIES:
    j = 0
    for f in VEHICLE_TYPES:
        prob += ce[r][f] == EFnew_param[r][r][j]*n[r][f]
        j += 1


In [14]:
# #Constraint 2: Decrease total emissions by D(s) for each state 
# for r in COUNTIES: 
#     prob += lpSum(ce[r][f] for f in VEHICLE_TYPES) <= W_param[r][0]
prob += lpSum(ce[r][f] for f in VEHICLE_TYPES for r in COUNTIES) <= ((1-D) * lpSum(W_param[r][0] for r in COUNTIES))

#### Commentary on Constraints 3,4
The idea here is that the count of a specific vehicle (FFV, BEV) cannot exceed the respective maximum theoretical count. Maybe we can do something here with the rural/urban debate of a county (e.g. Cook County case). Maybe combine the population density and the cars/person or something - to account for vehicle density. Perhaps something like (urban vs rural counties - e.g. urban has 1.5x more vehicles per capita?). We can actually compute this from county vehicle totals / population and interpolate the results.

In [15]:
e85_vi = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/e85_vi.csv')
efuels_vi = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/efuels_vi.csv')
e85_vi = e85_vi[['STATE','NAME','CENSUSAREA','e85_area']]
e85_vi = e85_vi.rename(columns={'STATE': 'state', 'NAME': 'county','CENSUSAREA':'census_area'})
efuels_vi = efuels_vi[['STATE','NAME','CENSUSAREA','efuels_area']]
efuels_vi = efuels_vi.rename(columns={'STATE': 'state', 'NAME': 'county','CENSUSAREA':'census_area'})

In [16]:
e85_vi['e85_area'] = e85_vi['e85_area'] / 100
efuels_vi['efuels_area'] = efuels_vi['efuels_area'] / 100

In [17]:
e85vi_param = e85_vi.groupby('county')['e85_area'].apply(list).to_dict()
efuelsvi_param = efuels_vi.groupby('county')['efuels_area'].apply(list).to_dict()

In [18]:
# vehicle_count_Aitkin_FFV = 22605.0 -- but should be zero.
# e85vi_param['Aitkin'][0] * T_param['Aitkin'][0]

In [19]:
#Constraints 3: E85 Viability index
for r in COUNTIES:
    prob += lpSum(n[r]['FFV']) <= (e85vi_param[r][0] * T_param[r][0]) 

In [20]:
#Constraint 4: EV Viability index
for r in COUNTIES:
    prob += lpSum(n[r]['BEV']) <= (efuelsvi_param[r][0] * T_param[r][0]) 

In [21]:
#Constraint 5: Total annual fuel consumption per county equals fuel consumption of each vehicle in that county
for r in COUNTIES:
    for f in VEHICLE_TYPES:
        prob += fc[r][f]  - (CFnew_param[r][f][0] * n[r][f]* TM_param[r][f][0]) == 0

In [22]:
#Constraint 6: Operating cost= cost of fuel * sum of fuel consumption
for r in COUNTIES:
    for f in VEHICLE_TYPES:
        prob += oc[r][f] - (CGnew_param[r][f][0] * fc[r][f]) == 0

In [23]:
# Constraint 7: Total number of all vehicle types v should be equal to the total number of vehicles in county r 
for r in COUNTIES:
    prob += lpSum(n[r][f] for f in VEHICLE_TYPES) == T_param[r][0]

In [24]:
# Constraint 8: Total annual cost of vehicle v in county r = operating cost + capital cost
for r in COUNTIES: 
    for f in VEHICLE_TYPES:
        prob += tac[r][f] == oc[r][f] + (CC_param[r][f][0] * n[r][f])

In [25]:
LpSolverDefault.msg=1

In [26]:
# prob.writeLP("BioFuelsModel.lp")

In [27]:
prob.solve()

1

In [44]:
print("Decision Variable Values")
for v in prob.variables():
        print(v.name, "=", v.varValue)

Decision Variable Values
GHG_emissions_new_Aitkin_BEV = 0.0
GHG_emissions_new_Aitkin_FFV = 0.0
GHG_emissions_new_Aitkin_SIDI_ICE = 103216630000.0
GHG_emissions_new_Alameda_BEV = 1509873600000.0
GHG_emissions_new_Alameda_FFV = 820606970000.0
GHG_emissions_new_Alameda_SIDI_ICE = 0.0
GHG_emissions_new_Alpine_BEV = 1504303700.0
GHG_emissions_new_Alpine_FFV = 4.8496207e-07
GHG_emissions_new_Alpine_SIDI_ICE = 2130418100.0
GHG_emissions_new_Amador_BEV = 47672211000.0
GHG_emissions_new_Amador_FFV = 0.0
GHG_emissions_new_Amador_SIDI_ICE = 24976876000.0
GHG_emissions_new_Anderson_BEV = 22395733000.0
GHG_emissions_new_Anderson_FFV = 0.0
GHG_emissions_new_Anderson_SIDI_ICE = 180888800000.0
GHG_emissions_new_Andrews_BEV = 13190182.0
GHG_emissions_new_Andrews_FFV = 0.0
GHG_emissions_new_Andrews_SIDI_ICE = 35891633000.0
GHG_emissions_new_Angelina_BEV = 49368549000.0
GHG_emissions_new_Angelina_FFV = 0.0
GHG_emissions_new_Angelina_SIDI_ICE = 285966620000.0
GHG_emissions_new_Anoka_BEV = 1161096600000.0


In [29]:
print("The Min Value = ",value(prob.objective))

The Min Value =  395557790269.8116


In [30]:
keys, values = [[] for i in range(2)]
for v in prob.variables():
    keys.append(v.name)
    values.append(v.varValue)
results = dict(zip(keys, values))

In [31]:
for i in COUNTIES:
    COUNTIES[COUNTIES.index(i)] = i.replace(" ", "_")

In [32]:
ghg_vis, oc_vis, tac_vis, tfu_vis, vc_vis, vc_vis_BEV, vc_vis_FFV, vc_vis_SIDI = [[] for i in range(8)]

for r in COUNTIES:
    a_subset = {key: value for key, value in results.items() if r in key}
    ghg_vis.append(a_subset['GHG_emissions_new_' + r + '_BEV'] + a_subset['GHG_emissions_new_' + r + '_FFV'] + a_subset['GHG_emissions_new_' + r + '_SIDI_ICE'])
    oc_vis.append(a_subset['annual_operating_cost_' + r + '_BEV'] + a_subset['annual_operating_cost_' + r + '_FFV'] + a_subset['annual_operating_cost_' + r + '_SIDI_ICE'])
    tac_vis.append(a_subset['total_annual_cost_' + r + '_BEV'] + a_subset['total_annual_cost_' + r + '_FFV'] + a_subset['total_annual_cost_' + r + '_SIDI_ICE'])
    tfu_vis.append(a_subset['total_fuel_use_' + r + '_BEV'] + a_subset['total_fuel_use_' + r + '_FFV'] + a_subset['total_fuel_use_' + r + '_SIDI_ICE'])
    vc_vis.append(a_subset['vehicle_count_' + r + '_BEV'] + a_subset['vehicle_count_' + r + '_FFV'] + a_subset['vehicle_count_' + r + '_SIDI_ICE'])
    vc_vis_BEV.append(a_subset['vehicle_count_' + r + '_BEV'])
    vc_vis_FFV.append(a_subset['vehicle_count_' + r + '_FFV'])
    vc_vis_SIDI.append(a_subset['vehicle_count_' + r + '_SIDI_ICE'])

In [33]:
Visual = pd.DataFrame(zip(COUNTIES, ghg_vis, oc_vis, tac_vis, tfu_vis, vc_vis, vc_vis_BEV, vc_vis_FFV, vc_vis_SIDI),
              columns=['County','ghg_vis','oc_vis', 'tac_vis','tfu_vis','vc_vis','vc_vis_BEV','vc_vis_FFV','vc_vis_SIDI'])
Visual[Visual['County'] == 'San_Francisco']

,County,ghg_vis,oc_vis,tac_vis,tfu_vis,vc_vis,vc_vis_BEV,vc_vis_FFV,vc_vis_SIDI
37,San_Francisco,1.550179e+12,121634010.6,2.695319e+09,52349552.0,454532.9973,446841.06,7691.9373,0.0


In [34]:
column_names = ["County", "annual_ghg_emissions", "annual_operating_cost","total_annual_cost","annual_fuel_use", "total_vehicle_count",'vehicle_split_BEV','vehicle_split_FFV','vehicle_split_SIDI']
Visual.columns = column_names
Visual['vehicle_pc_BEV'] = (Visual['vehicle_split_BEV']/Visual['total_vehicle_count']*100).round(1)
Visual['vehicle_pc_FFV'] = (Visual['vehicle_split_FFV']/Visual['total_vehicle_count']*100).round(1)
Visual['vehicle_pc_SIDI'] = (Visual['vehicle_split_SIDI']/Visual['total_vehicle_count']*100).round(1)

In [35]:
# Visual[Visual['County'] == 'San_Francisco']
Visual[Visual['County'] == 'Lake_of_the_Woods']


,County,annual_ghg_emissions,annual_operating_cost,total_annual_cost,annual_fuel_use,total_vehicle_count,vehicle_split_BEV,vehicle_split_FFV,vehicle_split_SIDI,vehicle_pc_BEV,vehicle_pc_FFV,vehicle_pc_SIDI
96,Lake_of_the_Woods,2.330584e+10,3053890.9,39996436.0,1745080.5,5644.0,0.0,0.0,5644.0,0.0,0.0,100.0


In [36]:
# Visual['County'] = Visual['County'].str.split('_').apply(lambda x: x[0])
# Visual['County'] = Visual['County'] + '_' + Visual['state']

In [37]:
Visual[Visual['County'] == 'Siskiyou']


,County,annual_ghg_emissions,annual_operating_cost,total_annual_cost,annual_fuel_use,total_vehicle_count,vehicle_split_BEV,vehicle_split_FFV,vehicle_split_SIDI,vehicle_pc_BEV,vehicle_pc_FFV,vehicle_pc_SIDI
46,Siskiyou,4.277202e+10,20000915.51,215163035.0,7324357.49,30059.9997,1774.5487,0.0,28285.451,5.9,0.0,94.1


In [38]:
county_renaming_engine = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/county_renaming_engine.csv')
county_renaming_engine[county_renaming_engine['County']=='Lake_of_the_Woods']

,County,State
96,Lake_of_the_Woods,MN


In [39]:
Visual = Visual.merge(county_renaming_engine)
Visual.shape

(399, 13)

In [40]:
# Visual.iloc[Visual['County'] == 'Lake_of_the_Woods']
Visual['County'] = Visual['County'].str.replace('_',' ')
Visual.to_csv('data/visual_df.csv')

In [41]:
# Visual[Visual['State'] == 'CA']


# # if county includes selected_states then append a _ right before it
# # else just append _state

# Visual['mythbusters']=np.where(Visual['County'] 'BEV', CC['cost_minus_rebate']/8.5, CC['cost_minus_rebate']/5.5)

In [42]:
# Visual.head(50)

In [43]:
Visual.groupby('State').sum()

,annual_ghg_emissions,annual_operating_cost,total_annual_cost,annual_fuel_use,total_vehicle_count,vehicle_split_BEV,vehicle_split_FFV,vehicle_split_SIDI,vehicle_pc_BEV,vehicle_pc_FFV,vehicle_pc_SIDI
State,,,,,,,,,,,
CA,8.971984e+13,1.297268e+10,1.900354e+11,5.464592e+09,2.944056e+07,1.740778e+07,4.815020e+06,7.217758e+06,2261.5,306.4,3232.6
MN,3.134754e+13,2.935910e+09,4.862215e+10,1.611954e+09,7.400001e+06,3.971235e+06,1.714841e+06,1.713925e+06,1468.1,2342.8,4888.7
TX,8.889314e+13,5.997070e+09,1.569002e+11,4.389880e+09,2.492282e+07,1.277459e+07,3.461412e+06,8.686824e+06,2686.8,853.3,21859.5
